In [5]:
#（1）参照BiGram代码，实现TriGram模型的设计，在199801.txt数据集上完成TriGram模型的学习，并提供使用训练好的TriGram模型生成一句话的功能。

In [6]:
#检查并定义工作目录
import os
print(os.getcwd())
os.chdir('C:/Users/86184/Desktop/文本分析')

C:\Users\86184\Desktop\文本分析


In [7]:
from collections import defaultdict
import numpy as np
import heapq
import random

import re  # 导入正则表达式模块

class BiGram:
    def __init__(self, topk=10000):
        self.i2w = {}
        self.w2i = {}
        self.M = None
        self.topk = topk
        return None
    
    def sort_dict(self,d):
        sorted_d = sorted(d.items(), key = lambda item: item[1], reverse = True)
        return sorted_d
    
    def train(self, trainset):
        unkown = '<UKN>'
        start = '<S>'
        end = '<E>'
        self.i2w[0]=unkown
        self.i2w[1]=start
        self.i2w[2]=end
        totalcount = 0   #总词数
        wordcount = defaultdict(int)  #词频
        wordcount[start] = len(trainset)  #<s>句子开始标记数
        wordcount[end] = len(trainset)   #<e>句子结束标记数
        print("词频统计中...")
        #词频统计
        for sentence in trainset:
            for word in sentence:
                wordcount[word]+=1
        
        #汇总总词数
        for _,v in wordcount.items():
            totalcount+=v
        
        
        #选取topk高频词，同时获得 索引 与 词 的mapping 
        print("生成词表...")
        i = 3
        for w,_ in self.sort_dict(wordcount):
            if i < self.topk+3:
                self.i2w[i]=w
            i+=1
        
        #获得 词 与 索引 的mapping 
        for i,w in self.i2w.items():
            self.w2i[w] = i
        #print(self.w2i)
        
        #获得保留词的词频
        wordcount_filter = defaultdict(int)
        for word,v in wordcount.items():
            #print(word)
            if word in self.w2i:
                wordcount_filter[self.w2i[word]]=v
            else:
                wordcount_filter[self.w2i['<UKN>']]+=v
        #wordcount_filter[1]=len(trainset) #补充<s>
        #wordcount_filter[2]=len(trainset)
        
        #print(wordcount_filter)
                
        voc_len = len(self.i2w)
        
        print("计算BiGram概率...")
        self.M = np.zeros((voc_len,voc_len)) #初始化概率矩阵
        
        #统计二元组合出现次数
        print("统计二元组出现次数...")
        for sentence in trainset:
            sentence_full = ['<S>'] + sentence + ['<E>']
            sentence_idx = []
            for word in sentence_full:
                if word not in self.w2i:
                    sentence_idx.append(self.w2i['<UKN>'])
                else:
                    sentence_idx.append(self.w2i[word])

            for i in range(len(sentence_idx)-1):
                p1 = sentence_idx[i]
                p2 = sentence_idx[i+1]
                self.M[p1][p2]+=1
        
        #计算二元组条件概率
        print("计算二元组条件概率...")
        for i in range(voc_len):
            for j in range(voc_len):
                #print(self.M[i][j]+1, wordcount_filter[i], (self.M[i][j]+1)*wordcount_filter[i], (wordcount_filter[i]+wordlist_len))
                self.M[i][j]=(1.0*(self.M[i][j]+1)*wordcount_filter[i])/(wordcount_filter[i]+voc_len)
                #if self.M[i][j]!=0:
                #    print(self.M[i][j])
        #print(self.M)
        
        return None
        
    def perplexity(self, testset):
        '''
        计算困惑度
        '''
        sent_perp = [] #存储每句话的困惑度数值
        for sentence in testset:
            #句子序列添加句首和句尾标签，并将词转换成索引序列
            sentence_full = ['<S>'] + sentence + ['<E>']
            sentence_idx = []
            for word in sentence_full:
                if word not in self.w2i:
                    sentence_idx.append(self.w2i['<UKN>'])
                else:
                    sentence_idx.append(self.w2i[word])
            #计算每个句子的联合概率
            prob = 1
            for i in range(len(sentence_idx)-1):
                p1 = sentence_idx[i]
                p2 = sentence_idx[i+1]
                #print(self.M[p1][2])
                prob*=1/self.M[p1][2]
            
            sent_perp.append(np.power(prob,1/len(sentence_full)))#困惑度计算
        return np.mean(sent_perp)#返回句子困惑度的均值
            
    def infer(self):
        '''
        生成语句
        '''
        word_sequence = []
        #根据句首标签生成下一个位置的词
        start_idx = self.w2i['<S>'] 
        prob_list = self.M[start_idx].tolist()
        #print(prob_list[:100])
        #选择条件概率最大的前五个候选词，从中随机选择一个，增加多样性
        top5_idx = heapq.nlargest(5,range(len(prob_list)),prob_list.__getitem__)
        word_idx = top5_idx[random.randint(0,4)]
        #print(word_idx,self.i2w[word_idx])
        word_sequence.append(self.i2w[word_idx])
        start_idx = word_idx
        #直到遇到句尾标记停止，否则循环继续
        while(word_idx!=self.w2i['<E>']):
            prob_list = self.M[start_idx].tolist()
            #print(prob_list[:100])
            top5_idx = heapq.nlargest(5,range(len(prob_list)),prob_list.__getitem__)
            word_idx = top5_idx[random.randint(0,4)]
            #print(word_idx,self.i2w[word_idx])
            word_sequence.append(self.i2w[word_idx])
            start_idx = word_idx
        return ' '.join(word_sequence)

In [21]:
sentences = []
punctions = ['！', '。', '？', ',', '.', '!', '?']  # 添加更多需要的标点符号
with open('199801.txt', "r", encoding="utf-8") as f:
    c = 0
    for line in f:
        if c > 10000:
            break
        c += 1
        ls = line.strip().split('  ')
        line_segs = []
        candidate_sentence = []
        w_num = 0
        for i in range(1, len(ls)):
            if ls[i].endswith('/w') and ls[i][0] in punctions:
                w_num += 1
                if len(candidate_sentence) >= 1:
                    sentences.append(candidate_sentence)
                    candidate_sentence = []
            else:
                end_idx = ls[i].index('/')
                word = ls[i][:end_idx]
                # 检查是否只包含中文字符
                if all(u'\u4e00' <= c <= u'\u9fff' for c in word):
                    candidate_sentence.append(word)
        if len(candidate_sentence) > 0:
            sentences.append(candidate_sentence)
print("总句子数：", len(sentences))

# 假设 BiGram 类已经根据需要进行了修改，以支持中文处理和生成
split_rate = 0.8
topk = 10000

bigram = BiGram(topk)

split_rate = int(split_rate * len(sentences))
bigram.train(sentences[:split_rate])
print('已完成BiGram构建...')

print('BiGram困惑度：', bigram.perplexity(sentences[split_rate:]))

# 生成句子时过滤掉非中文字符
sentence = bigram.infer()
# 确保只打印中文字符
print('生成的句子：', ''.join([c for c in sentence if u'\u4e00' <= c <= u'\u9fff']))

总句子数： 19724
词频统计中...
生成词表...
计算BiGram概率...
统计二元组出现次数...
计算二元组条件概率...
已完成BiGram构建...
BiGram困惑度： 32.2570681851328
生成的句子： 这就有在新时期的


In [8]:
sentences = []
punctions = ['！','。','?','']
with open('199801.txt',"r",encoding="utf-8") as f:
    #c = 0
    for line in f:
        #if c>10000:
        #    break
        #c+=1
        ls = line.strip().split('  ')
        line_segs = []
        candidate_sentence = []
        w_num = 0
        for i in range(1,len(ls)):
            if ls[i].endswith('/w') and ls[i][0] in punctions:
                w_num+=1
                if len(candidate_sentence)>=1:
                    sentences.append(candidate_sentence)
                    candidate_sentence = []
            else:
                end_idx = ls[i].index('/')
                candidate_sentence.append(ls[i][:end_idx])
        if len(candidate_sentence)>0:
            sentences.append(candidate_sentence)
print("总句子数：",len(sentences))

split_rate = 0.8 
topk = 10000

bigram = BiGram(topk)

split_rate = int(split_rate*len(sentences))
bigram.train(sentences[:split_rate])
print('已完成BiGram构建...')

print('BiGram困惑度：',bigram.perplexity(sentences[split_rate:]))

#print(bigram.M[0][0])
sentence = bigram.infer() 
print('生成的句子:',sentence)

总句子数： 44473
词频统计中...
生成词表...
计算BiGram概率...
统计二元组出现次数...
计算二元组条件概率...
已完成BiGram构建...
BiGram困惑度： 13.342639884517247
生成的句子: 本报 记者 陈 运鹏 在 新 技术 的 一 种 <UKN> 的 <E>


In [16]:
#方法二：没有参考BiGram代码，靠个人实现
import re
import random
from collections import defaultdict, Counter

# 读取文件内容
def read_corpus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 清洗和预处理文本
def preprocess_text(text):
     # 使用正则表达式去除英文字母、标点符号和数字，只保留中文字符和空格
    text = re.sub(r'[^\w\s]', '', text)  # 去除标点符号
    text = re.sub(r'[a-zA-Z0-9]', '', text)  # 去除英文字母和数字
    # 只保留中文字符和空格
    text = ''.join(filter(lambda x: x in ' ' or '\u4e00' <= x <= '\u9fff', text))
    return text.split()

# 构建TriGram模型
def build_trigram_model(corpus):
    if len(corpus) < 3:
        raise ValueError("Corpus must contain at least three words to build a trigram model.")
    
    n_grams = defaultdict(Counter)
    bigram_counts = Counter()
    
    for i in range(len(corpus) - 2):
        bigram = (corpus[i], corpus[i + 1])
        bigram_counts[bigram] += 1
        n_grams[bigram][corpus[i + 2]] += 1
    
    return n_grams, bigram_counts

# 使用模型生成一句话
def generate_sentence(trigram_model, bigram_counts, num_words=10):
    sentence = []
    start_bigram = random.choice(list(bigram_counts.keys()))
    sentence.append(start_bigram[0])
    sentence.append(start_bigram[1])
    
    for _ in range(num_words - 2):
        if (sentence[-2], sentence[-1]) not in trigram_model:
            break
        next_word = random.choices(list(trigram_model[(sentence[-2], sentence[-1])].keys()), list(trigram_model[(sentence[-2], sentence[-1])].values()))[0]
        sentence.append(next_word)
    return ' '.join(sentence)

def calculate_perplexity(trigram_model, bigram_counts, corpus):
    num_words = len(corpus)
    if num_words < 3:
        return float('inf')  # 如果少于3个词，困惑度无穷大

    joint_probability = 1.0
    # 计算开始的bigram的概率
    start_bigram_count = bigram_counts.get((corpus[0], corpus[1]), 0)
    if start_bigram_count == 0:
        return float('inf')  # 如果开始的bigram计数为0，困惑度无穷大
    joint_probability *= start_bigram_count
    
    for i in range(1, num_words - 1):
        bigram = (corpus[i - 1], corpus[i])
        trigram = (bigram[0], bigram[1], corpus[i + 1])
        
        bigram_count = bigram_counts[bigram]
        trigram_count = trigram_model[bigram].get(trigram[2], 0)
        
        if bigram_count == 0 or trigram_count == 0:
            return float('inf')  # 如果bigram或trigram计数为0，困惑度无穷大
        
        # 计算条件概率，并更新联合概率
        conditional_probability = trigram_count / bigram_count
        joint_probability *= conditional_probability
    
    # 应用困惑度公式
    if joint_probability == 0:
        return float('inf')  # 如果联合概率为0，困惑度无穷大
    perplexity = joint_probability ** (-1.0 / num_words)
    return perplexity

# 主函数
def main():
    file_path = 'C:/Users/86184/Desktop/文本分析/199801.txt'
    corpus = read_corpus(file_path)
    corpus = preprocess_text(corpus)
    
    if len(corpus) < 3:
        raise ValueError("Corpus must contain at least three words to build a trigram model.")
        
    trigram_model, bigram_counts = build_trigram_model(corpus)
    sentence = generate_sentence(trigram_model, bigram_counts, num_words=10)
    # 用result代替sentence，并将sentence中的空格删去
    result = sentence.replace(" ", "")
    print("生成的句子:", result)
        
    perplexity = calculate_perplexity(trigram_model, bigram_counts, corpus)
    print(f"困惑度: {perplexity}")
        
if __name__ == "__main__":
    main()

生成的句子: 当着一车乘客的时间你们去干什么
困惑度: inf


In [ ]:
#困惑度为无穷大（inf）说明在计算过程中遇到了零概率事件或者模型未能适当地预测数据集中的序列。
#说明该模型不够成熟，需要改进，虽然运行速度非常快，但准确性不够。今后还需要继续学习，在实践中总结经验。